# Load gene attributes

In [1]:
using CSV
using DataFrames
using Statistics

df = CSV.File("../KIDNEY/node_attributes_noNA.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
df = unique(df, :name)      # remove possible duplicated genes
genenames = df[!, :name]
first(df, 15)

Row,name,label,label_wo_outliers,label_cluster2_kmeans5,label_cluster5_kmeans5,label_cluster4_kmeans5,gene_length,transcript_count,gc_content,official_gene_symbol,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String,String3?,String3?,String3?,String3?,String3?,Float64?,Float64?,Float64?,String15?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,(clone tec14),missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
2,100 kDa coactivator,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
3,14-3-3 tau splice variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
4,3-beta-hydroxysteroid dehydrogese,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,3-mercaptopyruvate sulfurtransferase variant,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
6,3'-phosphoadenosine-5'-phosphosulfate synthase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
7,40S ribosomal protein S15a,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
8,5-aminoimidazole-4-carboxamide ribonucleotide formyltransferase,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
9,5'-3' exoribonuclease,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing


In [27]:
df = df[typeintersect.(colwise(eltype, df), Number) .!= Union{}]      # select only numeric attributes
for col in eachcol(df)                                                # fix missing and naN with mean value
    m = mean(collect(skipmissing(col)))
    std = mean(collect(skipmissing(col)))
    replace!(col, missing => m)
    replace!(col, NaN => m)
    convert.(Float64,col)
end
df = mapcols!(ByRow(Float64), df)                                  # convert Union{missing,Float64} to Float64 types

MethodError: MethodError: no method matching getindex(::DataFrame, ::DataFrame)
Closest candidates are:
  getindex(::AbstractDataFrame, !Matched::CartesianIndex{2}) at ~/.julia/packages/DataFrames/dgZn3/src/other/broadcasting.jl:3
  getindex(::AbstractDataFrame, !Matched::Integer, !Matched::Colon) at ~/.julia/packages/DataFrames/dgZn3/src/dataframerow/dataframerow.jl:210
  getindex(::AbstractDataFrame, !Matched::Integer, !Matched::Union{Colon, Regex, AbstractVector, All, Between, Cols, InvertedIndex}) at ~/.julia/packages/DataFrames/dgZn3/src/dataframerow/dataframerow.jl:208
  ...

### zscore attributes

In [12]:
using StatsBase
dt = fit(ZScoreTransform, Matrix(df), dims=2)
x = StatsBase.transform(dt, Matrix(df)) #|> DataFrame
df = DataFrame(x, names(df))
df[!, :name] = genenames

MethodError: MethodError: no method matching fit(::Type{ZScoreTransform}, ::Matrix{Any}; dims=2)
Closest candidates are:
  fit(!Matched::Type{Histogram}, ::Any...; kwargs...) at ~/.julia/packages/StatsBase/XgjIN/src/hist.jl:407
  fit(::Type{ZScoreTransform}, !Matched::AbstractVector{<:Real}; dims, center, scale) at ~/.julia/packages/StatsBase/XgjIN/src/transformations.jl:130
  fit(::Type{ZScoreTransform}, !Matched::AbstractMatrix{<:Real}; dims, center, scale) at ~/.julia/packages/StatsBase/XgjIN/src/transformations.jl:109

## Load labels

In [210]:
dfl = CSV.File("../KIDNEY/node_labels.csv", typemap=Dict(Int => Float64)) |> DataFrame    # read the attribute file
dfl = unique(dfl, :name)         # remove gene duplicates
dfl = filter(:most_freq => !ismissing, dfl)

Row,name,ACH-000159,ACH-000189,ACH-000234,ACH-000246,ACH-000250,ACH-000262,ACH-000272,ACH-000313,ACH-000317,ACH-000375,ACH-000385,ACH-000411,ACH-000433,ACH-000484,ACH-000495,ACH-000600,ACH-000649,ACH-000684,ACH-000709,ACH-000792,ACH-000907,ACH-001163,ACH-001194,ACH-001310,ACH-001398,ACH-001532,ACH-001687,ACH-001688,ACH-002150,ACH-002189,most_freq
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3?,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3
1,A1BG,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE
2,A1CF,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,NE,NE,NE,aNE,NE,NE,aNE,aNE
3,A2M,NE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE
4,A2ML1,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE
5,A3GALT2,NE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE
6,A4GALT,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,aNE,NE,NE,NE,aNE,aNE,aNE,aNE
7,A4GNT,NE,aNE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE
8,AAAS,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aE,aNE,aNE,aNE,aNE,aNE
9,AACS,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,aNE,NE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,aNE,NE,aNE,aNE,aNE,aNE,aNE,aNE


## Select labels

In [211]:
dfl = filter(row -> row.most_freq ∈ ["E","NE"], dfl)

Row,name,ACH-000159,ACH-000189,ACH-000234,ACH-000246,ACH-000250,ACH-000262,ACH-000272,ACH-000313,ACH-000317,ACH-000375,ACH-000385,ACH-000411,ACH-000433,ACH-000484,ACH-000495,ACH-000600,ACH-000649,ACH-000684,ACH-000709,ACH-000792,ACH-000907,ACH-001163,ACH-001194,ACH-001310,ACH-001398,ACH-001532,ACH-001687,ACH-001688,ACH-002150,ACH-002189,most_freq
,String15,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3?,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3,String3
1,A2ML1,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE,NE
2,AADAC,NE,NE,aNE,NE,NE,aNE,NE,aNE,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,NE,NE,NE,NE,NE
3,AADACL3,NE,NE,NE,NE,NE,NE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,aNE,aNE,NE,NE,aNE,aNE,aNE,aNE,NE,NE,NE,NE,NE,aNE,NE
4,AADACL4,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,aNE,NE,NE,NE
5,AAGAB,NE,NE,aNE,NE,NE,NE,NE,NE,NE,aNE,aNE,aNE,NE,NE,NE,NE,aNE,NE,aNE,NE,NE,aNE,NE,NE,NE,aNE,aNE,NE,NE,NE,NE
6,AARS1,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E,E
7,AASS,aNE,NE,NE,aNE,NE,aNE,aNE,NE,aNE,NE,NE,aNE,aNE,NE,aNE,aNE,NE,aNE,aNE,NE,NE,NE,aNE,NE,NE,NE,aNE,NE,NE,NE,NE
8,ABCA10,aNE,NE,NE,NE,aNE,NE,NE,aNE,aNE,aNE,NE,NE,aNE,NE,NE,NE,aNE,NE,aNE,aNE,NE,NE,NE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE
9,ABCA7,NE,NE,aNE,aNE,NE,NE,aNE,NE,aNE,NE,NE,NE,NE,aNE,aNE,NE,aNE,NE,aNE,aNE,NE,NE,NE,NE,aNE,NE,NE,NE,NE,NE,NE


In [212]:
genenames = dfl[!, :name]
dfall = innerjoin(dfl,df; on=:name)[!, Not(r"ACH-")]         # join labels and attributes dataframes

Row,name,most_freq,label,label_wo_outliers,label_cluster2_kmeans5,label_cluster5_kmeans5,label_cluster4_kmeans5,gene_length,transcript_count,gc_content,official_gene_symbol,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole
,String15,String3,String3?,String3?,String3?,String3?,String3?,Float64?,Float64?,Float64?,String15?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,A2ML1,NE,CS6,CS6,CS6,CS6,CS6,64380.0,9.0,44.23,A2ML1,0.0394261,7.0,missing,0.0,4.0,2.0,3.0,121.0,missing,missing,31.0,4.0,159.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,AADAC,NE,CS7,CS7,CS7,CS6,CS5,14415.0,2.0,35.3,AADAC,0.0270464,1.0,missing,0.2,7.0,3.0,2.0,1.0,missing,3.0,13.0,3.0,184.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,AAGAB,NE,CS6,CS6,CS5,CS6,CS6,54528.0,7.0,39.47,AAGAB,19.5814,4.0,-0.395414,27.7,1.0,1.0,3.0,32.0,missing,missing,84.0,7.0,345.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
4,AARS1,E,CS0,CS0,CS0,CS0,CS0,37724.0,42.0,49.58,AARS1,missing,2.0,missing,44.0,9.0,8.0,5.0,108.0,1.0,4.0,78.0,6.0,324.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing
5,AASS,NE,CS6,CS6,CS6,CS6,CS6,79725.0,27.0,36.53,AASS,6.92464,14.0,-1.14935,14.0,9.0,5.0,6.0,53.0,2.0,3.0,72.0,1.0,340.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
6,ABCA10,NE,CS6,CS6,CS6,CS6,CS5,97632.0,13.0,36.13,ABCA10,1.16924,2.0,missing,0.7,4.0,2.0,2.0,5.0,1.0,3.0,54.0,4.0,53.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,ABCA7,NE,CS6,CS6,CS7,CS6,CS4,25575.0,18.0,60.53,ABCA7,7.26944,10.0,0.257249,1.1,8.0,28.0,12.0,20.0,1.0,3.0,87.0,5.0,204.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,ABCB7,E,CS0,CS0,CS0,CS1,CS0,105684.0,17.0,36.0,ABCB7,10.0881,9.0,0.0694002,9.7,6.0,9.0,3.0,139.0,1.0,5.0,95.0,4.0,338.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,ABCC1,NE,CS6,CS6,CS4,CS6,CS6,193419.0,7.0,48.47,ABCC1,9.32908,20.0,0.345639,0.4,13.0,21.0,9.0,141.0,5.0,20.0,72.0,5.0,277.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


# Load embedding

In [213]:
dfe = CSV.File("../KIDNEY/embeddings/PPI_Node2Vec_64.csv") |> DataFrame    # read the attribute file
dfall = innerjoin(dfall,dfe; on=:name)               # join labels/attributes dataframe with embedding

Row,name,most_freq,label,label_wo_outliers,label_cluster2_kmeans5,label_cluster5_kmeans5,label_cluster4_kmeans5,gene_length,transcript_count,gc_content,official_gene_symbol,Gtex_kidney,gene_disease_ass_count,oncodb_expression,HPA_kidney_match_symbol,mf,bp,cc,biogrid,kegg,reactome,ucsc_tfbs,up_tissue,orth_count,Mitochondrion,Nucleus,Endosome,Golgi apparatus,Cytosol,Plasma membrane,Endoplasmic reticulum,Cytoskeleton,Peroxisome,Lysosome,Extracellular space,Vacuole,Node2Vec_1,Node2Vec_2,Node2Vec_3,Node2Vec_4,Node2Vec_5,Node2Vec_6,Node2Vec_7,Node2Vec_8,Node2Vec_9,Node2Vec_10,Node2Vec_11,Node2Vec_12,Node2Vec_13,Node2Vec_14,Node2Vec_15,Node2Vec_16,Node2Vec_17,Node2Vec_18,Node2Vec_19,Node2Vec_20,Node2Vec_21,Node2Vec_22,Node2Vec_23,Node2Vec_24,Node2Vec_25,Node2Vec_26,Node2Vec_27,Node2Vec_28,Node2Vec_29,Node2Vec_30,Node2Vec_31,Node2Vec_32,Node2Vec_33,Node2Vec_34,Node2Vec_35,Node2Vec_36,Node2Vec_37,Node2Vec_38,Node2Vec_39,Node2Vec_40,Node2Vec_41,Node2Vec_42,Node2Vec_43,Node2Vec_44,Node2Vec_45,Node2Vec_46,Node2Vec_47,Node2Vec_48,Node2Vec_49,Node2Vec_50,Node2Vec_51,Node2Vec_52,Node2Vec_53,Node2Vec_54,Node2Vec_55,Node2Vec_56,Node2Vec_57,Node2Vec_58,Node2Vec_59,Node2Vec_60,Node2Vec_61,Node2Vec_62,Node2Vec_63,Node2Vec_64
,String15,String3,String3?,String3?,String3?,String3?,String3?,Float64?,Float64?,Float64?,String15?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,FNTA,E,CS0,CS0,CS0,CS1,CS0,51594.0,14.0,42.58,FNTA,15.1028,missing,0.332371,36.4,12.0,8.0,6.0,78.0,1.0,18.0,24.0,7.0,324.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0327755,0.84089,0.762798,-0.474988,0.375936,-0.453812,-0.882691,0.440906,-0.878124,-0.533896,-0.621145,-0.330792,-0.764699,0.298258,-0.163851,0.84922,1.10704,0.424903,0.638736,1.12484,0.359124,1.56491,-0.204588,-0.172318,0.181068,-0.754538,1.38996,1.04818,-0.605823,-0.513815,-0.21651,-0.83456,-0.530819,-1.30349,0.353133,0.521479,1.06585,-0.496735,0.561891,0.394384,-0.441063,0.400256,0.0466632,0.518116,0.640591,0.58238,0.449851,0.737192,-0.104968,-0.751855,-0.239735,0.285988,-0.0820393,-0.139406,0.196502,0.663146,0.158625,0.452507,-0.653603,-1.04916,1.30081,0.165904,0.00732472,-0.758322
2,ACVR1,NE,CS6,CS6,CS5,CS6,CS5,140079.0,26.0,40.42,ACVR1,8.47767,28.0,-0.07676,26.6,17.0,50.0,8.0,123.0,4.0,missing,145.0,2.0,257.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,-1.9554,0.210282,0.367882,-1.56708,-0.857939,0.62536,-0.870367,-0.319358,-1.63851,-0.667482,0.98611,-1.72713,-0.555335,-0.096736,0.910918,0.137879,0.822529,-0.558942,1.32688,0.258166,-0.912445,0.0193069,-0.232802,-0.604843,-0.581782,0.24931,-0.506253,0.69955,0.0970136,-1.24973,0.09171,-0.2101,0.27826,-0.191165,0.0811974,0.789027,0.682285,-0.969496,0.0484258,0.780362,-0.101702,-0.168606,0.425831,0.00610404,1.86621,1.21535,0.562664,0.0565809,0.312061,0.152257,-0.266875,-2.33547,-0.640375,0.371121,0.384757,0.496508,-1.29307,0.343799,0.0702841,-0.537535,-0.280513,1.06283,0.853472,-0.538201
3,PML,NE,CS6,CS6,CS7,CS6,CS5,53154.0,22.0,47.81,PML,12.9565,21.0,0.887473,8.1,13.0,72.0,14.0,353.0,7.0,29.0,82.0,6.0,208.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.441455,0.529117,0.249046,-1.49291,0.443501,-0.0286639,-0.672491,0.785152,-0.116008,0.665382,0.485512,-0.0766194,0.757544,-0.509678,-0.789928,0.510408,0.646292,-0.138827,-0.808072,0.300918,0.618837,-0.449619,0.602021,-0.24196

In [214]:
X = Matrix(dfall[!, Not(r"name|most_freq")])
y = vec(Matrix(DataFrames.select(dfall, [:most_freq])))
size(X), size(y)

((5661, 98), (5661,))

In [215]:
using ScikitLearn.CrossValidation: cross_val_predict
using DecisionTree

function confussionmatrix(predictions, labels)
   classes = vec(unique(labels))
   d = size(classes)[1]
   idx = Dict(zip(classes,Vector(1:d)))
   c = zeros(Int64, d,d)
   for i in 1:size(labels)[1]
       c[idx[labels[i]] ,idx[predictions[i]]] += 1 
   end 
   df = DataFrame(c, classes)
   df[!, :name] = classes
   return df
end
model = DecisionTree.RandomForestClassifier()
ŷ = cross_val_predict(model, X, y; cv=5)
y_targets = y
accuracy = sum(ŷ .== y_targets) / length(y_targets)
println("accuracy: $accuracy")
display(confussionmatrix(ŷ,y))

ArgumentError: ArgumentError: Package DecisionTree not found in current path:
- Run `import Pkg; Pkg.add("DecisionTree")` to install the DecisionTree package.


In [219]:
MLJBase.accuracy(ŷ, y)

LoadError: MethodError: no method matching (::MisclassificationRate)(::Vector{String3}, ::Vector{String3})